In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols, logit
from statsmodels.stats.anova import anova_lm


In [4]:
 #--- Step 0: Load CSV properly ---
file_path = 'C:/Users/USER/Desktop/Projects/Insurance Risk Analytics/Insurance-Risk-Analytics/data/processed_insurance_data.csv'
df = pd.read_csv(file_path, sep='|')

# Inspect first rows to confirm
print(df.head())
print(df.columns.tolist())

C:\Users\USER\AppData\Local\Temp\ipykernel_17988\3066445958.py:3: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='|')


   underwrittencoverid  policyid transactionmonth  isvatregistered  \
0               145249     12827       2015-03-01             True   
1               145249     12827       2015-05-01             True   
2               145249     12827       2015-07-01             True   
3               145255     12827       2015-05-01             True   
4               145255     12827       2015-07-01             True   

  citizenship          legaltype title language                 bank  \
0              Close Corporation    Mr  English  First National Bank   
1              Close Corporation    Mr  English  First National Bank   
2              Close Corporation    Mr  English  First National Bank   
3              Close Corporation    Mr  English  First National Bank   
4              Close Corporation    Mr  English  First National Bank   

       accounttype  ... covercategory   covertype            covergroup  \
0  Current account  ...    Windscreen  Windscreen  Comprehensive - Taxi

In [5]:
# --- Step 1: Create Metrics ---
# Claim Frequency: 1 if TotalClaims > 0 else 0
df['ClaimFrequency'] = (df['totalclaims'] > 0).astype(int)

# Claim Severity: Only for policies with claims
df['ClaimSeverity'] = df['totalclaims'] / df['ClaimFrequency'].replace(0, np.nan)

# Margin
df['Margin'] = df['totalpremium'] - df['totalclaims']


In [8]:
# --- Step 2: Hypothesis Testing (Memory-Efficient) ---

# Aggregate claim frequency and severity by postal code
zip_agg = df.groupby('postalcode').agg(
    ClaimFrequencyRate=('ClaimFrequency', 'mean'),
    ClaimSeverityMean=('ClaimSeverity', 'mean'),
    NumPolicies=('ClaimFrequency', 'count'),
    AvgMargin=('Margin', 'mean')
).reset_index()

# Optional: filter out postal codes with very few policies
zip_agg = zip_agg[zip_agg['NumPolicies'] >= 10]

import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

# --- Provinces ---
print("\n--- Provinces: Claim Severity ---")
prov_model = ols('ClaimSeverity ~ C(province)', data=df.dropna(subset=['ClaimSeverity'])).fit()
prov_anova = anova_lm(prov_model)
print(prov_anova)

print("\n--- Provinces: Claim Frequency ---")
prov_freq_model = ols('ClaimFrequency ~ C(province)', data=df).fit()
prov_freq_anova = anova_lm(prov_freq_model)
print(prov_freq_anova)

# --- Zip Codes (aggregated) ---
print("\n--- Zip Codes: Claim Severity ---")
zip_sev_model = ols('ClaimSeverityMean ~ 1', data=zip_agg).fit()  # overall mean
print(zip_agg[['postalcode', 'ClaimSeverityMean']].head())  # inspect

print("\n--- Zip Codes: Claim Frequency Rate ---")
zip_freq_model = ols('ClaimFrequencyRate ~ 1', data=zip_agg).fit()
print(zip_agg[['postalcode', 'ClaimFrequencyRate']].head())  # inspect

print("\n--- Zip Codes: Margin ---")
margin_model = ols('AvgMargin ~ 1', data=zip_agg).fit()
print(zip_agg[['postalcode', 'AvgMargin']].head())

# --- Gender ---
print("\n--- Gender: Claim Severity ---")
gender_model = ols('ClaimSeverity ~ C(gender)', data=df.dropna(subset=['ClaimSeverity'])).fit()
gender_anova = anova_lm(gender_model)
print(gender_anova)

print("\n--- Gender: Claim Frequency ---")
gender_freq_model = ols('ClaimFrequency ~ C(gender)', data=df).fit()
gender_freq_anova = anova_lm(gender_freq_model)
print(gender_freq_anova)


--- Provinces: Claim Severity ---
                 df        sum_sq       mean_sq         F    PR(>F)
C(province)     8.0  5.730111e+10  7.162639e+09  4.830166  0.000006
Residual     2779.0  4.120971e+12  1.482897e+09       NaN       NaN

--- Provinces: Claim Frequency ---
                    df       sum_sq   mean_sq        F        PR(>F)
C(province)        8.0     0.289646  0.036206  13.0251  5.912100e-19
Residual     1000089.0  2779.938172  0.002780      NaN           NaN

--- Zip Codes: Claim Severity ---
   postalcode  ClaimSeverityMean
0           1       25631.945175
1           2       10314.216374
2           4                NaN
3           5       20737.881579
4           6        4314.298246

--- Zip Codes: Claim Frequency Rate ---
   postalcode  ClaimFrequencyRate
0           1            0.002247
1           2            0.004032
2           4            0.000000
3           5            0.010000
4           6            0.004545

--- Zip Codes: Margin ---
   postalcode

In [10]:
# --- Step 3: Automated Interpretation ---
def interpret_anova(anova_table, feature_name):
    p_val = anova_table['PR(>F)'][0]
    if p_val < 0.05:
        print(f"Reject H0 for {feature_name} (p = {p_val:.4f}) → significant effect.")
    else:
        print(f"Fail to reject H0 for {feature_name} (p = {p_val:.4f}) → no significant effect.")

interpret_anova(prov_anova, "Province Claim Severity")
interpret_anova(prov_freq_anova, "Province Claim Frequency")
interpret_anova(gender_anova, "Gender Claim Severity")
interpret_anova(gender_freq_anova, "Gender Claim Frequency")

print("\nZip Code metrics are aggregated; check the table above for differences.")

Reject H0 for Province Claim Severity (p = 0.0000) → significant effect.
Reject H0 for Province Claim Frequency (p = 0.0000) → significant effect.
Fail to reject H0 for Gender Claim Severity (p = 0.0899) → no significant effect.
Reject H0 for Gender Claim Frequency (p = 0.0266) → significant effect.

Zip Code metrics are aggregated; check the table above for differences.


C:\Users\USER\AppData\Local\Temp\ipykernel_17988\4127240761.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_val = anova_table['PR(>F)'][0]
